In [1]:
import os
import gym
import random
import numpy as np
import tensorflow as tf
from collections import deque

from skimage.color import rgb2gray
from skimage.transform import resize

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, Dense, Flatten

In [2]:
# 상태가 입력, 큐함수가 출력인 인공신경망 생성
class DQN(tf.keras.Model):
    def __init__(self, action_size, state_size):
        super(DQN, self).__init__()
        self.conv1 = Conv2D(32, (8, 8), strides=(4, 4), activation='relu',
                            input_shape=state_size)
        self.conv2 = Conv2D(64, (4, 4), strides=(2, 2), activation='relu')
        self.conv3 = Conv2D(64, (3, 3), strides=(1, 1), activation='relu')
        self.flatten = Flatten()
        self.fc = Dense(512, activation='relu')
        self.fc_out = Dense(action_size)

    def call(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.flatten(x)
        x = self.fc(x)
        q = self.fc_out(x)
        return q

In [3]:
# 브레이크아웃 예제에서의 DQN 에이전트
class DQNAgent:
    def __init__(self, action_size, state_size=(84, 84, 4)):
        self.render = False

        # 상태와 행동의 크기 정의
        self.state_size = state_size
        self.action_size = action_size

        # DQN 하이퍼파라미터
        self.discount_factor = 0.99
        self.learning_rate = 1e-4
        self.epsilon = 1.
        self.epsilon_start, self.epsilon_end = 1.0, 0.02
        self.exploration_steps = 1000000.
        self.epsilon_decay_step = self.epsilon_start - self.epsilon_end
        self.epsilon_decay_step /= self.exploration_steps
        self.batch_size = 32
        self.train_start = 50000
        self.update_target_rate = 10000

        # 리플레이 메모리, 최대 크기 100,000
        self.memory = deque(maxlen=100000)
        # 게임 시작 후 랜덤하게 움직이지 않는 것에 대한 옵션
        self.no_op_steps = 30

        """
        DQN에서 유의해야 할 점은 target model도 같은 형태로 생성해야 한다는 것이다.
        model과 target model을 생성하고 두 모델의 가중치를 통일하는 코드가 아래의 코드이다.
        """
        # 모델과 타깃 모델 생성
        self.model = DQN(action_size, state_size)
        self.target_model = DQN(action_size, state_size)
        
        """
        오류함수를 계산한 후 그 오류를 최소화하는 방향으로 모델을 업데이트해주는 optimizer를 사용해 훈련한다.
        아래의 코드는 optimizer를 선언하는 부분이다.
        이 때 clipnorm=10 인자는 그레이디언트 클리핑(gradient clipping)을 위한 인자이다.
        그레이디언트 클리핑은 그레이디언트(gradient)의 크기가 특정 값을 넘어가지 못하게 하는 것을 의미한다.
        그레이디언트의 크기가 너무 크면 모델의 가중치가 너무 급격하게 변화하면서 학습에 악영향을 미칠 수 있기 때문에
        이를 방지하기 위함이다.
        (후버로스 함수와 비슷한 역할을 수행한다. 후버로스와 그레이디언트 클리핑은 모두 강건한(robust)한 훈련은 가능하게 한다.)
        """
        self.optimizer = Adam(self.learning_rate, clipnorm=10.)
        
        # 타깃 모델 초기화
        self.update_target_model()

        self.avg_q_max, self.avg_loss = 0, 0

        self.writer = tf.summary.create_file_writer('summary/spaceinvader_dqn')
        self.model_path = os.path.join(os.getcwd(), 'save_model', 'model')

    # 타깃 모델을 모델의 가중치로 업데이트
    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

    """
    모델을 생성했으면 에이전트는 모델을 통해 행동을 선택할 수 잇다.
    행동의 선택은 케라스의 model() 함수를 이용해 출력한 큐함수를 통해 선택한다.
    큐함수에 대해 에이전트는 입실론 탐욕 정책을 통해 행동을 선택한다.
    이때 모델에 들어가는 입력은 원래 픽셀 값인 0~255 사이의 값이 아니라 0~1 사이의 값이 되도록 한다.
    """
    
    # 입실론 탐욕 정책으로 행동 선택
    def get_action(self, history):
        history = np.float32(history / 255.0)
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        else:
            q_value = self.model(history)
            return np.argmax(q_value[0])
        
    """1
   에이전트가 환경에서 한 스텝을 진행하고 환경으로부터 정보를 얻으면 샘플로서 리플레이 메모리에 저장한다.
   리플레이 메모리에는 현재 히스토리와 행동, 보상, 다음 히스토리, 목숨을 잃었는지의 정보를 넣는다.
   """
    # 샘플 <s, a, r, s'>을 리플레이 메모리에 저장
    def append_sample(self, history, action, reward, next_history, dead):
        self.memory.append((history, action, reward, next_history, dead))

    # 텐서보드에 학습 정보를 기록
    def draw_tensorboard(self, score, step, episode):
        with self.writer.as_default():
            tf.summary.scalar('Total Reward/Episode', score, step=episode)
            tf.summary.scalar('Average Max Q/Episode',
                              self.avg_q_max / float(step), step=episode)
            tf.summary.scalar('Duration/Episode', step, step=episode)
            tf.summary.scalar('Average Loss/Episode',
                              self.avg_loss / float(step), step=episode)

    """
    리플레이 메모리에 샘플을 추가했으면 에이전트는 리플레이 메모리를 이용해 학습을 진행한다.
    학습은 첫 50000 스텝이 지난 후부터 진행하므로 메모리의 사이즈가 50000 이하일 때는 학습하지 않는다.
    학습을 시작하면 입실론을 타임스텝마다 감소시킨다.
    """
    # 리플레이 메모리에서 무작위로 추출한 배치로 모델 학습
    def train_model(self):
        if self.epsilon > self.epsilon_end:
            self.epsilon -= self.epsilon_decay_step

        # 메모리에서 배치 크기만큼 무작위로 샘플 추출
        batch = random.sample(self.memory, self.batch_size)

        """
        추출한 샘플들을 학습에 필요한 형태로 처리해주기 위해 아래와 같은 코드가 필요하다.
        리플레이 메모리에 넣어주었던 순서대로 history, actions, rewards, next_history, done을 각각 추출해서 numpy array 형태로 변환한다.
        """
        history = np.array([sample[0][0] / 255. for sample in batch],
                           dtype=np.float32)
        actions = np.array([sample[1] for sample in batch])
        rewards = np.array([sample[2] for sample in batch])
        next_history = np.array([sample[3][0] / 255. for sample in batch],
                                dtype=np.float32)
        dones = np.array([sample[4] for sample in batch])

        # 학습 파라메터
        model_params = self.model.trainable_variables
        with tf.GradientTape() as tape:
            # 현재 상태에 대한 모델의 큐함수
            predicts = self.model(history)
            one_hot_action = tf.one_hot(actions, self.action_size)
            predicts = tf.reduce_sum(one_hot_action * predicts, axis=1)

            # 다음 상태에 대한 타깃 모델의 큐함수
            target_predicts = self.target_model(next_history)

            """
            후버로스(Huber Loss)라는 오류함수를 사용.
            후버로스는 -1에서 1 사이의 구간에서는 2차함수이며 그 밖의 구간에서는 1차함수인 오류함수이다.
            MSE 오류함수의 경우에는 모든 구간에서 2차함수이기 때문에 큰 값의 오류에 대해서 민감하게 학습될 수 밖에 없다.
            후버로스는 절댓값이 1이 넘어가는 구간에 대해서 1차함수를 적용하여 학습을 안정화 시킨다.
            이 때문에 MSE 오류함수대신 후버로스 오류함수를 사용한다.
            (그레디언트 클리핑과 비슷한 역할을 수행한다. 오류함수의 값이 커지면 그레이디언트 크기 또한 커질 수 있는데,
            후버로스가 이를 방지한다. 후버로스와 그레이디언트 클리핑은 모두 강건한(robust)한 훈련은 가능하게 한다.)
            아래의 코드 두 블럭이 후버로스 함수를 구현한 코드이다.
            """
            # 벨만 최적 방정식을 구성하기 위한 타깃과 큐함수의 최대 값 계산
            max_q = np.amax(target_predicts, axis=1)
            targets = rewards + (1 - dones) * self.discount_factor * max_q

            # 후버로스 계산
            error = tf.abs(targets - predicts)
            quadratic_part = tf.clip_by_value(error, 0.0, 1.0)
            linear_part = error - quadratic_part
            loss = tf.reduce_mean(0.5 * tf.square(quadratic_part) + linear_part)

            self.avg_loss += loss.numpy()

        # 오류함수를 줄이는 방향으로 모델 업데이트
        grads = tape.gradient(loss, model_params)
        self.optimizer.apply_gradients(zip(grads, model_params))



In [4]:
# 학습속도를 높이기 위해 흑백화면으로 전처리
def pre_processing(observe):
    processed_observe = np.uint8(
        resize(rgb2gray(observe), (84, 84), mode='constant') * 255)
    return processed_observe

In [ ]:
if __name__ == "__main__":
    # 환경과 DQN 에이전트 생성
    env = gym.make('SpaceInvaders-v4')
    agent = DQNAgent(action_size=6)

    global_step = 0
    score_avg = 0
    score_max = 0

    # 불필요한 행동을 없애주기 위한 딕셔너리 선언
    # action_dict = {0:1, 1:2, 2:3, 3:3}
    action_dict = {0:1, 1:2, 2:3, 3:4, 4:5, 5:5}

    num_episode = 50000
    for e in range(num_episode):
        done = False
        dead = False

        step, score, start_life = 0, 0, 3
        # env 초기화
        observe = env.reset()

        # 랜덤으로 뽑힌 값 만큼의 프레임동안 움직이지 않음
        for _ in range(random.randint(1, agent.no_op_steps)):
            observe, _, _, _ = env.step(1)

        # 프레임을 전처리 한 후 4개의 상태를 쌓아서 입력값으로 사용.
        """
        아래의 코드는 에피소드의 처음 시작 히스토리를 만드는 부분이다.
        첫 상태에서는 4개의 화면을 모을 수 없기 때문에 첫 상태를 똑같이 4개 복사해서 하나의 히스토리로 만든다.
        np.stack 함수에서 axis는 어떤 축에 대해 state를 쌓을 것인가에 대한 변수이다.
        state가 [84,84,1]의 형태를 가지기 때문에 1에 해당하는 축에 대해 state를 쌓아줘야 한다.
        따라서 axis=2라고 설정한다.
        """
        state = pre_processing(observe)
        history = np.stack((state, state, state, state), axis=2)
        history = np.reshape([history], (1, 84, 84, 4))

        while not done:
            if agent.render:
                env.render()
            global_step += 1
            step += 1

            # 바로 전 history를 입력으로 받아 행동을 선택
            action = agent.get_action(history)
            # 1: 정지, 2: 왼쪽, 3: 오른쪽
            real_action = action_dict[action]

            # 죽었을 때 시작하기 위해 발사 행동을 함
            if dead:
                action, real_action, dead = 0, 1, False

            """
            get_action 함수를 통해 행동을 선택하면 에이전트는 환경에서 그 행동으로 한 스텝을 진행한다.
            환경은 에이전트에게 다음 observe의 보상, 에피소드가 끝났는지에 대한 정보, 목숨이 몇 개 남았는지에 대한 정보를 제공한다.
            """
            # 선택한 행동으로 환경에서 한 타임스텝 진행
            observe, reward, done, info = env.step(real_action)

            # 각 타임스텝마다 상태 전처리
            next_state = pre_processing(observe)
            next_state = np.reshape([next_state], (1, 84, 84, 1))
            next_history = np.append(next_state, history[:, :, :, :3], axis=3)

            agent.avg_q_max += np.amax(agent.model(np.float32(history / 255.))[0])

            
            if start_life > info['lives']:
                dead = True
                start_life = info['lives']

            score += reward
            
           
            reward = np.clip(reward, -1., 1.)
            
            # 샘플 <s, a, r, s'>을 리플레이 메모리에 저장 후 학습
            agent.append_sample(history, action, reward, next_history, dead)

            # 리플레이 메모리 크기가 정해놓은 수치에 도달한 시점부터 모델 학습 시작
            if len(agent.memory) >= agent.train_start:
                agent.train_model()
                # 일정 시간마다 타겟모델을 모델의 가중치로 업데이트
                if global_step % agent.update_target_rate == 0:
                    agent.update_target_model()

            if dead:
                history = np.stack((next_state, next_state,
                                    next_state, next_state), axis=2)
                history = np.reshape([history], (1, 84, 84, 4))
            else:
                history = next_history

            if done:
                # 각 에피소드 당 학습 정보를 기록
                if global_step > agent.train_start:
                    agent.draw_tensorboard(score, step, e)

                score_avg = 0.9 * score_avg + 0.1 * score if score_avg != 0 else score
                score_max = score if score > score_max else score_max

                log = "episode: {:5d} | ".format(e)
                log += "score: {:4.1f} | ".format(score)
                log += "score max : {:4.1f} | ".format(score_max)
                log += "score avg: {:4.1f} | ".format(score_avg)
                log += "memory length: {:5d} | ".format(len(agent.memory))
                log += "epsilon: {:.3f} | ".format(agent.epsilon)
                log += "q avg : {:3.2f} | ".format(agent.avg_q_max / float(step))
                log += "avg loss : {:3.2f}".format(agent.avg_loss / float(step))
                print(log)

                agent.avg_q_max, agent.avg_loss = 0, 0

        # 1000 에피소드마다 모델 저장
        if e % 1000 == 0:
            agent.model.save_weights("./save_model/model", save_format="tf")
            
            
        


episode:     0 | score: 110.0 | score max : 110.0 | score avg: 110.0 | memory length:   704 | epsilon: 1.000 | q avg : 0.03 | avg loss : 0.00
episode:     1 | score: 65.0 | score max : 110.0 | score avg: 105.5 | memory length:  1077 | epsilon: 1.000 | q avg : 0.03 | avg loss : 0.00
episode:     2 | score: 355.0 | score max : 355.0 | score avg: 130.4 | memory length:  1851 | epsilon: 1.000 | q avg : 0.03 | avg loss : 0.00
episode:     3 | score: 160.0 | score max : 355.0 | score avg: 133.4 | memory length:  2615 | epsilon: 1.000 | q avg : 0.03 | avg loss : 0.00
episode:     4 | score: 65.0 | score max : 355.0 | score avg: 126.6 | memory length:  3259 | epsilon: 1.000 | q avg : 0.03 | avg loss : 0.00
episode:     5 | score: 105.0 | score max : 355.0 | score avg: 124.4 | memory length:  3843 | epsilon: 1.000 | q avg : 0.03 | avg loss : 0.00
episode:     6 | score: 80.0 | score max : 355.0 | score avg: 120.0 | memory length:  4214 | epsilon: 1.000 | q avg : 0.03 | avg loss : 0.00
episode: 

episode:    58 | score: 105.0 | score max : 490.0 | score avg: 171.8 | memory length: 43498 | epsilon: 1.000 | q avg : 0.03 | avg loss : 0.00
episode:    59 | score: 135.0 | score max : 490.0 | score avg: 168.1 | memory length: 44237 | epsilon: 1.000 | q avg : 0.03 | avg loss : 0.00
episode:    60 | score: 210.0 | score max : 490.0 | score avg: 172.3 | memory length: 45053 | epsilon: 1.000 | q avg : 0.03 | avg loss : 0.00
episode:    61 | score: 210.0 | score max : 490.0 | score avg: 176.1 | memory length: 45867 | epsilon: 1.000 | q avg : 0.03 | avg loss : 0.00
episode:    62 | score: 255.0 | score max : 490.0 | score avg: 184.0 | memory length: 46836 | epsilon: 1.000 | q avg : 0.03 | avg loss : 0.00
episode:    63 | score: 255.0 | score max : 490.0 | score avg: 191.1 | memory length: 47663 | epsilon: 1.000 | q avg : 0.03 | avg loss : 0.00
episode:    64 | score: 30.0 | score max : 490.0 | score avg: 175.0 | memory length: 48035 | epsilon: 1.000 | q avg : 0.04 | avg loss : 0.00
episode

episode:   116 | score: 50.0 | score max : 490.0 | score avg: 140.6 | memory length: 85234 | epsilon: 0.965 | q avg : 0.19 | avg loss : 0.00
episode:   117 | score: 180.0 | score max : 490.0 | score avg: 144.6 | memory length: 86050 | epsilon: 0.965 | q avg : 0.18 | avg loss : 0.00
episode:   118 | score: 15.0 | score max : 490.0 | score avg: 131.6 | memory length: 86482 | epsilon: 0.964 | q avg : 0.18 | avg loss : 0.00
episode:   119 | score: 410.0 | score max : 490.0 | score avg: 159.4 | memory length: 87291 | epsilon: 0.963 | q avg : 0.17 | avg loss : 0.00
episode:   120 | score: 50.0 | score max : 490.0 | score avg: 148.5 | memory length: 87690 | epsilon: 0.963 | q avg : 0.17 | avg loss : 0.00
episode:   121 | score: 50.0 | score max : 490.0 | score avg: 138.6 | memory length: 88113 | epsilon: 0.963 | q avg : 0.20 | avg loss : 0.00
episode:   122 | score: 240.0 | score max : 490.0 | score avg: 148.8 | memory length: 89027 | epsilon: 0.962 | q avg : 0.18 | avg loss : 0.00
episode:  

episode:   174 | score: 125.0 | score max : 730.0 | score avg: 205.3 | memory length: 100000 | epsilon: 0.922 | q avg : 0.34 | avg loss : 0.01
episode:   175 | score: 120.0 | score max : 730.0 | score avg: 196.7 | memory length: 100000 | epsilon: 0.922 | q avg : 0.27 | avg loss : 0.01
episode:   176 | score: 485.0 | score max : 730.0 | score avg: 225.6 | memory length: 100000 | epsilon: 0.920 | q avg : 0.44 | avg loss : 0.01
episode:   177 | score: 225.0 | score max : 730.0 | score avg: 225.5 | memory length: 100000 | epsilon: 0.919 | q avg : 0.31 | avg loss : 0.01
episode:   178 | score: 210.0 | score max : 730.0 | score avg: 224.0 | memory length: 100000 | epsilon: 0.919 | q avg : 0.40 | avg loss : 0.01
episode:   179 | score: 210.0 | score max : 730.0 | score avg: 222.6 | memory length: 100000 | epsilon: 0.918 | q avg : 0.35 | avg loss : 0.01
episode:   180 | score: 210.0 | score max : 730.0 | score avg: 221.3 | memory length: 100000 | epsilon: 0.917 | q avg : 0.36 | avg loss : 0.01

episode:   232 | score: 105.0 | score max : 730.0 | score avg: 132.1 | memory length: 100000 | epsilon: 0.882 | q avg : 0.55 | avg loss : 0.01
episode:   233 | score: 135.0 | score max : 730.0 | score avg: 132.4 | memory length: 100000 | epsilon: 0.881 | q avg : 0.67 | avg loss : 0.01
episode:   234 | score: 410.0 | score max : 730.0 | score avg: 160.1 | memory length: 100000 | epsilon: 0.880 | q avg : 0.69 | avg loss : 0.01
episode:   235 | score: 335.0 | score max : 730.0 | score avg: 177.6 | memory length: 100000 | epsilon: 0.879 | q avg : 0.72 | avg loss : 0.01
episode:   236 | score: 210.0 | score max : 730.0 | score avg: 180.9 | memory length: 100000 | epsilon: 0.879 | q avg : 0.62 | avg loss : 0.01
episode:   237 | score: 210.0 | score max : 730.0 | score avg: 183.8 | memory length: 100000 | epsilon: 0.878 | q avg : 0.76 | avg loss : 0.01
episode:   238 | score: 180.0 | score max : 730.0 | score avg: 183.4 | memory length: 100000 | epsilon: 0.877 | q avg : 0.57 | avg loss : 0.01

...

episode:  3596 | score: 30.0 | score max : 905.0 | score avg: 226.3 | memory length: 100000 | epsilon: 0.020 | q avg : 4.31 | avg loss : 0.02
episode:  3597 | score: 175.0 | score max : 905.0 | score avg: 221.2 | memory length: 100000 | epsilon: 0.020 | q avg : 4.23 | avg loss : 0.01
episode:  3598 | score: 215.0 | score max : 905.0 | score avg: 220.6 | memory length: 100000 | epsilon: 0.020 | q avg : 4.06 | avg loss : 0.01
episode:  3599 | score: 395.0 | score max : 905.0 | score avg: 238.0 | memory length: 100000 | epsilon: 0.020 | q avg : 3.71 | avg loss : 0.01
episode:  3600 | score: 85.0 | score max : 905.0 | score avg: 222.7 | memory length: 100000 | epsilon: 0.020 | q avg : 4.53 | avg loss : 0.02
episode:  3601 | score: 170.0 | score max : 905.0 | score avg: 217.4 | memory length: 100000 | epsilon: 0.020 | q avg : 3.96 | avg loss : 0.01
episode:  3602 | score: 525.0 | score max : 905.0 | score avg: 248.2 | memory length: 100000 | epsilon: 0.020 | q avg : 3.55 | avg loss : 0.02
e

episode:  3654 | score: 140.0 | score max : 905.0 | score avg: 180.3 | memory length: 100000 | epsilon: 0.020 | q avg : 4.18 | avg loss : 0.01
episode:  3655 | score: 290.0 | score max : 905.0 | score avg: 191.3 | memory length: 100000 | epsilon: 0.020 | q avg : 3.52 | avg loss : 0.01
episode:  3656 | score: 520.0 | score max : 905.0 | score avg: 224.2 | memory length: 100000 | epsilon: 0.020 | q avg : 4.04 | avg loss : 0.01
episode:  3657 | score: 265.0 | score max : 905.0 | score avg: 228.3 | memory length: 100000 | epsilon: 0.020 | q avg : 4.01 | avg loss : 0.01
episode:  3658 | score: 410.0 | score max : 905.0 | score avg: 246.4 | memory length: 100000 | epsilon: 0.020 | q avg : 3.83 | avg loss : 0.01
episode:  3659 | score: 105.0 | score max : 905.0 | score avg: 232.3 | memory length: 100000 | epsilon: 0.020 | q avg : 4.61 | avg loss : 0.02
episode:  3660 | score: 80.0 | score max : 905.0 | score avg: 217.1 | memory length: 100000 | epsilon: 0.020 | q avg : 4.51 | avg loss : 0.01


episode:  3712 | score: 110.0 | score max : 905.0 | score avg: 209.8 | memory length: 100000 | epsilon: 0.020 | q avg : 4.10 | avg loss : 0.02
episode:  3713 | score: 170.0 | score max : 905.0 | score avg: 205.8 | memory length: 100000 | epsilon: 0.020 | q avg : 4.07 | avg loss : 0.02
episode:  3714 | score: 300.0 | score max : 905.0 | score avg: 215.2 | memory length: 100000 | epsilon: 0.020 | q avg : 4.11 | avg loss : 0.01
episode:  3715 | score: 65.0 | score max : 905.0 | score avg: 200.2 | memory length: 100000 | epsilon: 0.020 | q avg : 4.12 | avg loss : 0.01
episode:  3716 | score: 80.0 | score max : 905.0 | score avg: 188.2 | memory length: 100000 | epsilon: 0.020 | q avg : 4.43 | avg loss : 0.01
episode:  3717 | score: 160.0 | score max : 905.0 | score avg: 185.4 | memory length: 100000 | epsilon: 0.020 | q avg : 4.39 | avg loss : 0.02
episode:  3718 | score: 300.0 | score max : 905.0 | score avg: 196.8 | memory length: 100000 | epsilon: 0.020 | q avg : 3.84 | avg loss : 0.02
e

episode:  3770 | score: 265.0 | score max : 905.0 | score avg: 220.4 | memory length: 100000 | epsilon: 0.020 | q avg : 3.81 | avg loss : 0.01
episode:  3771 | score: 170.0 | score max : 905.0 | score avg: 215.4 | memory length: 100000 | epsilon: 0.020 | q avg : 4.10 | avg loss : 0.02
episode:  3772 | score: 240.0 | score max : 905.0 | score avg: 217.9 | memory length: 100000 | epsilon: 0.020 | q avg : 3.66 | avg loss : 0.02
episode:  3773 | score: 185.0 | score max : 905.0 | score avg: 214.6 | memory length: 100000 | epsilon: 0.020 | q avg : 3.53 | avg loss : 0.02
episode:  3774 | score: 240.0 | score max : 905.0 | score avg: 217.1 | memory length: 100000 | epsilon: 0.020 | q avg : 3.68 | avg loss : 0.02
episode:  3775 | score: 555.0 | score max : 905.0 | score avg: 250.9 | memory length: 100000 | epsilon: 0.020 | q avg : 3.99 | avg loss : 0.02
episode:  3776 | score: 215.0 | score max : 905.0 | score avg: 247.3 | memory length: 100000 | epsilon: 0.020 | q avg : 3.90 | avg loss : 0.02

episode:  3828 | score: 120.0 | score max : 905.0 | score avg: 209.3 | memory length: 100000 | epsilon: 0.020 | q avg : 4.38 | avg loss : 0.02
episode:  3829 | score: 180.0 | score max : 905.0 | score avg: 206.4 | memory length: 100000 | epsilon: 0.020 | q avg : 4.02 | avg loss : 0.02
episode:  3830 | score: 145.0 | score max : 905.0 | score avg: 200.3 | memory length: 100000 | epsilon: 0.020 | q avg : 4.20 | avg loss : 0.02
episode:  3831 | score: 145.0 | score max : 905.0 | score avg: 194.7 | memory length: 100000 | epsilon: 0.020 | q avg : 4.40 | avg loss : 0.02
episode:  3832 | score: 255.0 | score max : 905.0 | score avg: 200.8 | memory length: 100000 | epsilon: 0.020 | q avg : 3.75 | avg loss : 0.02
episode:  3833 | score: 420.0 | score max : 905.0 | score avg: 222.7 | memory length: 100000 | epsilon: 0.020 | q avg : 3.97 | avg loss : 0.02
episode:  3834 | score: 245.0 | score max : 905.0 | score avg: 224.9 | memory length: 100000 | epsilon: 0.020 | q avg : 4.09 | avg loss : 0.02